In [2]:
import numpy as np
import sympy as sy
import matplotlib.pyplot as plt
import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app
from tqdm import tqdm
import random

In [3]:
n_particles = 1
system = mm.System()
mass   = 14.007 * unit.amu # masa del átomo N
system.addParticle(mass)

force = mm.CustomExternalForce('Eo*(C*x^4-x^2)+M*x + (K/2.0)*(y^2+z^2)')
force.addGlobalParameter('K', 35 * unit.kilocalories_per_mole/unit.angstrom**2)
force.addGlobalParameter('Eo', 6.0 * unit.kilocalories_per_mole/unit.angstrom**2)
force.addGlobalParameter('C', 0.4 * 1/unit.angstrom**2)
force.addGlobalParameter('M', 0.25 * unit.kilocalories_per_mole/unit.angstrom)
force.addParticle(0, [])
system.addForce(force)

0

In [ ]:
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temps = [300,315,330,345,360,375,390]
rep = range(len(temps))
pressure    = None

step_size       = 0.01*unit.picoseconds
num_steps       = 20000
saving_period   = 100
num_steps_saved = int(num_steps/saving_period)

friction   = 1.0/unit.picosecond

platform_name = 'CPU'  
platform = mm.Platform.getPlatformByName(platform_name)

contexto = []


for T in temps:
    integrator_T = mm.LangevinIntegrator(T, friction, step_size)
    times = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.picoseconds)
    positions  = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms)
    velocities = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms/unit.picosecond)
    potential_energies = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)
    kinetic_energies   = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)

    initial_positions  = np.zeros([n_particles, 3], np.float32) * unit.nanometers
    initial_velocities = None 

    context_T = mm.Context(system, integrator_T, platform)
    context_T.setPositions(initial_positions_T)
    context_T.setVelocitiesToTemperature(T)

    state_T = context_T.getState(getEnergy=True, getPositions=True, getVelocities=True)
    times_T[0] = state_T.getTime()
    positions_T[0] = state_T.getPositions()[0]
    velocities_T[0] = state_T.getVelocities()[0]
    kinetic_energies_T[0]=state_T.getKineticEnergy()
    potential_energies_T[0]=state_T.getPotentialEnergy()

    for ii in tqdm(range(num_steps_saved)):
        context_T.getIntegrator().step(saving_period)
        state_T = context_T.getState(getEnergy=True, getPositions=True, getVelocities=True)
        times_T[ii] = state_T.getTime()
        positions_T[ii] = state_T.getPositions()[0]
        velocities_T[ii] = state_T.getVelocities()[0]
        kinetic_energies_T[ii]=state_T.getKineticEnergy()
        potential_energies_T[ii]=state_T.getPotentialEnergy()
 

In [ ]:
reps = []
reps.append(int(input("Escribe T inicial:")))
reps.append(int(input("Escribe la temperatura máxima:")))
reps.append(int(input("Escribe el incremento de T entre réplicas:")))

reps = np.arange(reps[0],reps[1]+reps[2],reps[2])
reps = list(reps)*unit.kelvin

print(reps)

In [ ]:
reps = []
reps.append(int(input("Escribe T inicial:")))
reps.append(int(input("Escribe la temperatura máxima:")))
reps.append(int(input("Escribe el incremento de T entre réplicas:")))

reps = np.arange(reps[0],reps[1]+reps[2],reps[2])
reps = list(reps)*unit.kelvin
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
pressure    = None

num_steps       = 20000
saving_period   = 100
num_steps_saved = int(num_steps/saving_period)

friction   = 1.0/unit.picosecond

platform_name = 'CPU'  
platform = mm.Platform.getPlatformByName(platform_name)

In [4]:
reps = [300,301,302,303,304,305]*unit.kelvin

In [5]:
step_size       = 0.01*unit.picoseconds
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
pressure    = None

num_steps       = 20000
saving_period   = 100
num_steps_saved = int(num_steps/saving_period)

friction   = 1.0/unit.picosecond
intg = {}
for ii in range(len(reps)):
    for T in reps:
        intg["integrator{0}".format(ii)] = mm.LangevinIntegrator(T, friction, step_size)
    
ts = {}
pos = {}
vel = {}
pot_en = {}
kin_en = {}
init_pos = {}
init_vel = {}
contxt = {}
sts = {}

for ii in range(len(reps)):
    ts["times{0}".format(ii)] = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.picoseconds)
    pos["positions{0}".format(ii)]  = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms)
    vel["velocities{0}".format(ii)] = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms/unit.picosecond)
    pot_en["potential_energies{0}".format(ii)] = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)
    kin_en["kinetic_energies{0}".format(ii)] = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)
    init_pos["initial_positions{0}".format(ii)] = np.zeros([n_particles, 3], np.float32) * unit.nanometers
    init_vel["initial_velocities{0}".format(ii)] = None 


In [10]:
for ii in range(len(reps)):
    contxt["context{0}".format(ii)] = mm.Context(system, integrator[ii], platform)
    sts["state{0}".format(ii)]= context[ii].getState(getEnergy=True, getPositions=True, getVelocities=True)

KeyError: 0

In [6]:
intg

{'integrator0': <simtk.openmm.openmm.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::LangevinIntegrator *' at 0x7f25be20b4b0> >,
 'integrator1': <simtk.openmm.openmm.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::LangevinIntegrator *' at 0x7f25be20b2a0> >,
 'integrator2': <simtk.openmm.openmm.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::LangevinIntegrator *' at 0x7f25be20b5a0> >,
 'integrator3': <simtk.openmm.openmm.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::LangevinIntegrator *' at 0x7f25be20b5d0> >,
 'integrator4': <simtk.openmm.openmm.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::LangevinIntegrator *' at 0x7f25be20b600> >,
 'integrator5': <simtk.openmm.openmm.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::LangevinIntegrator *' at 0x7f25be20b630> >}